# Run all train, all test and upload

In [ ]:
import time
import numpy as np
from tools.csp import generate_projection, generate_eye, extract_feature
from tools.filters import load_filterbank
from sklearn.model_selection import KFold
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from tools.data import DreemDatasets
from preprocessing import Compose, ExtractBands, ExtractSpectrum
from models.riemannian_multiscale import riemannian_multiscale
import pandas as pd

In [ ]:
fs = 50.  # sampling frequency
NO_channels = 7  # number of EEG channels
NO_riem = int(NO_channels * NO_channels + 1) / 2  # Total number of CSP feature per band and timewindow
bw = np.array([2, 4, 8, 13, 22])
ftype = 'butter'  # 'fir', 'butter'
forder = 2  # 4
filter_bank = load_filterbank(bw, fs, order=forder, max_freq=23, ftype=ftype)  # get filterbank coeffs
time_windows_flt = np.array([[0, 30],
                             [15, 30],
                             [10, 25],
                             [5, 20],
                             [0, 15],
                             [15, 25],
                             [10, 20],
                             [5, 15],
                             [0, 10]
]) * fs

#time_windows = time_windows[0:1]  # use only largest timewindow


riem_opt = "No_Adaptation"  # {"Riemann","Riemann_Euclid","Whitened_Euclid","No_Adaptation"}
rho = 0.1

time_windows = time_windows_flt.astype(int)
NO_bands = filter_bank.shape[0]
NO_csp = 24  # Total number of CSP feature per band and timewindow
useCSP = False

In [ ]:
def get_data(path, train=True):
    if train:
        for i in range(7):
            if i==0:
                feature_0 = np.load("dataset/"+path+"/train_split/eeg_" + str(i + 1) + ".npy")
                X = np.zeros((7, feature_0.shape[0], feature_0.shape[1]))
                X[0] = feature_0
                del feature_0
            else:
                X[i] = np.load("dataset/"+path+"/train_split/eeg_" + str(i + 1) + ".npy")
        Y = np.load("dataset/"+path+"/train_split/targets.npy")
        X = X.transpose((1, 0, 2))
        return(X, Y)
    else:
        for i in range(7):
            if i==0:
                feature_0 = np.load("dataset/"+path+"/test/eeg_" + str(i + 1) + ".npy")
                X = np.zeros((7, feature_0.shape[0], feature_0.shape[1]))
                X[0] = feature_0
                del feature_0
            else:
                X[i] = np.load("dataset/"+path+"/test/eeg_" + str(i + 1) + ".npy")
        X = X.transpose((1, 0, 2))
        return(X)
    
def get_data_extra_data_eeg(path, train=True):
    if train:
        for i in range(7):
            if i==0:
                feature_0 = np.load("dataset/"+path+"/extra_eeg/train_split/eeg_" + str(i + 1) + ".npy").transpose((1, 0, 2))
                X = np.zeros((7, feature_0.shape[0], feature_0.shape[1]*feature_0.shape[2]))
                X[i] = feature_0.reshape(feature_0.shape[0], feature_0.shape[1]*feature_0.shape[2])
                del feature_0
            else:
                feature_0 = np.load("dataset/"+path+"/extra_eeg/train_split/eeg_" + str(i + 1) + ".npy").transpose((1, 0, 2))
                X[i] = feature_0.reshape(feature_0.shape[0], feature_0.shape[1]*feature_0.shape[2])
                del feature_0
        Y = np.load("dataset/"+path+"/train_split/targets.npy")
        X = X.transpose((1, 0, 2))
        return(X, Y)
    else:
        for i in range(7):
            if i==0:
                feature_0 = np.load("dataset/"+path+"/extra_eeg/test/eeg_" + str(i + 1) + ".npy").transpose((1, 0, 2))
                X = np.zeros((7, feature_0.shape[0], feature_0.shape[1]*feature_0.shape[2]))
                X[i] = feature_0.reshape(feature_0.shape[0], feature_0.shape[1]*feature_0.shape[2])
                del feature_0
            else:
                feature_0 = np.load("dataset/"+path+"/extra_eeg/test/eeg_" + str(i + 1) + ".npy").transpose((1, 0, 2))
                X[i] = feature_0.reshape(feature_0.shape[0], feature_0.shape[1]*feature_0.shape[2])
                del feature_0
        X = X.transpose((1, 0, 2))
        return(X)

def get_extra_data(path, train=True):
    if train:
        use_datasets = ["accelerometer_x","accelerometer_y","accelerometer_z","pulse_oximeter_infrared"]
        for i in range(4):
            if i==0:
                feature_0 = np.load("dataset/"+path+"/train_split/" + use_datasets[i] + ".npy").transpose((1, 0, 2))
                print(feature_0.shape)
                X = np.zeros((4, feature_0.shape[0], feature_0.shape[1]*feature_0.shape[2]))
                X[i] = feature_0.reshape(feature_0.shape[0], feature_0.shape[1]*feature_0.shape[2])
                del feature_0
            else:
                feature_0 = np.load("dataset/"+path+"/train_split/" + use_datasets[i] + ".npy").transpose((1, 0, 2))
                X[i] = feature_0.reshape(feature_0.shape[0], feature_0.shape[1]*feature_0.shape[2])
                del feature_0
        Y = np.load("dataset/"+path+"/train_split/targets.npy")
        X = X.transpose((1, 0, 2))
        return(X, Y)
    else:
        use_datasets = ["accelerometer_x","accelerometer_y","accelerometer_z","pulse_oximeter_infrared"]
        for i in range(4):
            if i==0:
                feature_0 = np.load("dataset/"+path+"/test/" + use_datasets[i] + ".npy").transpose((1, 0, 2))
                print(feature_0.shape)
                X = np.zeros((4, feature_0.shape[0], feature_0.shape[1]*feature_0.shape[2]))
                X[i] = feature_0.reshape(feature_0.shape[0], feature_0.shape[1]*feature_0.shape[2])
                del feature_0
            else:
                feature_0 = np.load("dataset/"+path+"/test/" + use_datasets[i] + ".npy").transpose((1, 0, 2))
                X[i] = feature_0.reshape(feature_0.shape[0], feature_0.shape[1]*feature_0.shape[2])
                del feature_0
        X = X.transpose((1, 0, 2))
        return(X)
    
def get_data_val(path):
    for i in range(7):
        if i==0:
            feature_0 = np.load("dataset/"+path+"/val_split/eeg_" + str(i + 1) + ".npy")
            X = np.zeros((7, feature_0.shape[0], feature_0.shape[1]))
            X[0] = feature_0
            del feature_0
        else:
            X[i] = np.load("dataset/"+path+"/val_split/eeg_" + str(i + 1) + ".npy")
    Y = np.load("dataset/"+path+"/val_split/targets.npy")
    X = X.transpose((1, 0, 2))
    return(X, Y)

def get_extrat_data_val_eeg(path):
    for i in range(7):
        if i==0:
            feature_0 = np.load("dataset/"+path+"/extra_eeg/val_split/eeg_" + str(i + 1) + ".npy").transpose((1, 0, 2))
            X = np.zeros((7, feature_0.shape[0], feature_0.shape[1]*feature_0.shape[2]))
            X[i] = feature_0.reshape(feature_0.shape[0], feature_0.shape[1]*feature_0.shape[2])
            del feature_0
        else:
            feature_0 = np.load("dataset/"+path+"/extra_eeg/val_split/eeg_" + str(i + 1) + ".npy").transpose((1, 0, 2))
            X[i] = feature_0.reshape(feature_0.shape[0], feature_0.shape[1]*feature_0.shape[2])
            del feature_0
    Y = np.load("dataset/"+path+"/extra_eeg/val_split/targets.npy")
    X = X.transpose((1, 0, 2))
    return(X, Y)

def get_extra_data_val(path):
    use_datasets = ["accelerometer_x","accelerometer_y","accelerometer_z","pulse_oximeter_infrared"]
    for i in range(4):
        if i==0:
            feature_0 = np.load("dataset/"+path+"/val_split/" + use_datasets[i] + ".npy").transpose((1, 0, 2))
            print(feature_0.shape)
            X = np.zeros((4, feature_0.shape[0], feature_0.shape[1]*feature_0.shape[2]))
            X[i] = feature_0.reshape(feature_0.shape[0], feature_0.shape[1]*feature_0.shape[2])
            del feature_0
        else:
            feature_0 = np.load("dataset/"+path+"/val_split/" + use_datasets[i] + ".npy").transpose((1, 0, 2))
            X[i] = feature_0.reshape(feature_0.shape[0], feature_0.shape[1]*feature_0.shape[2])
            del feature_0
    Y = np.load("dataset/"+path+"/val_split/targets.npy")
    X = X.transpose((1, 0, 2))
    return(X, Y)

path = "balanced"
train_data, train_label = get_data(path, train = True)
path = "all"
test_data, test_label = get_data(path)


## Features simples

In [ ]:
path = "balanced"
train_extra_data, train_extra_label = get_extra_data(path, train = True)
path = "all"
test_extra_data,_ = get_extra_data(path)
train_extra_data = train_extra_data.reshape(-1, 4*16)
test_extra_data = test_extra_data.reshape(-1,  4*16)

In [ ]:
start = time.time()

clf = RandomForestClassifier(n_estimators=700,max_features='auto', random_state=0)
clf.fit(train_extra_data, train_label)
print("trained")

labels_pred = clf.predict(test_extra_data)
print("predicted")

cm = confusion_matrix(test_label, labels_pred)
acc = accuracy_score(test_label, labels_pred)
f1 = f1_score(test_label, labels_pred, average='macro')
print(cm, acc, f1)


print("time : ", time.time() - start)


In [ ]:
path = "balanced"
train_extra_data_eeg, train_extra_label = get_data_extra_data_eeg(path, train = True)
path = "all"
test_extra_data_eeg,_ = get_data_extra_data_eeg(path)
print(test_extra_data_eeg.shape)
train_extra_data_eeg = train_extra_data_eeg.reshape(-1, 7*24)
test_extra_data_eeg = test_extra_data_eeg.reshape(-1,  7*24)

In [ ]:
start = time.time()

clf = RandomForestClassifier(n_estimators=700,max_features='auto', random_state=0)
clf.fit(train_extra_data_eeg, train_label)
print("trained")

labels_pred = clf.predict(test_extra_data_eeg)
print("predicted")

cm = confusion_matrix(test_label, labels_pred)
acc = accuracy_score(test_label, labels_pred)
f1 = f1_score(test_label, labels_pred, average='macro')
print(cm, acc, f1)

print("time : ", time.time() - start)


In [ ]:
all_final_features = np.concatenate((train_extra_data_eeg, train_extra_data), axis= 1)
all_final_features_test = np.concatenate((test_extra_data_eeg, test_extra_data), axis= 1)
print(all_final_features.shape)


In [ ]:
start = time.time()

clf = RandomForestClassifier(n_estimators=700,max_features='auto', random_state=0)
clf.fit(all_final_features, train_label)
print("trained")

labels_pred = clf.predict(all_final_features_test)
print("predicted")

cm = confusion_matrix(test_label, labels_pred)
acc = accuracy_score(test_label, labels_pred)
f1 = f1_score(test_label, labels_pred, average='macro')
print(cm, acc, f1)

print("time : ", time.time() - start)


## Avec CSP

In [ ]:
def get_features(data, label, time_windows, useCSP = True):
    if useCSP:
        w = generate_projection(data, label, NO_csp, filter_bank, time_windows, NO_classes=5)
    else:
        w = generate_eye(data, label, filter_bank, time_windows)
    feature_mat = extract_feature(data, w, filter_bank, time_windows)
    return(w, feature_mat)

In [ ]:
w, train_feat_CSP = get_features(train_data, train_label, time_windows, useCSP)
test_feature_CSP = extract_feature(test_data, w, filter_bank, time_windows)
#val_feature_CSP = extract_feature(val_data, w, filter_bank, time_windows)
del w

In [ ]:
features_CSP_train = train_feat_CSP
features_CSP_test = test_feature_CSP

In [ ]:
start = time.time()

clf = RandomForestClassifier(n_estimators=700,max_features='auto', random_state=0)
clf.fit(all_final_features, train_label)
print("trained")

labels_pred = clf.predict(all_final_features_test)
print("predicted")

cm = confusion_matrix(test_label, labels_pred)
acc = accuracy_score(test_label, labels_pred)
f1 = f1_score(test_label, labels_pred, average='macro')
print(cm, acc, f1)

print("time : ", time.time() - start)


In [ ]:

all_final_features = np.concatenate((features_CSP_train, train_extra_data), axis= 1)
all_final_features_test = np.concatenate((features_CSP_test, test_extra_data), axis= 1)
print(all_final_features.shape)

In [ ]:
start = time.time()

clf = RandomForestClassifier(n_estimators=700,max_features='auto', random_state=0)
clf.fit(all_final_features, train_label)
print("trained")

labels_pred = clf.predict(all_final_features_test)
print("predicted")

cm = confusion_matrix(test_label, labels_pred)
acc = accuracy_score(test_label, labels_pred)
f1 = f1_score(test_label, labels_pred, average='macro')
print(cm, acc, f1)

print("time : ", time.time() - start)


In [ ]:




all_final_features = np.concatenate((features_CSP_train, train_extra_data_eeg), axis= 1)
all_final_features_test = np.concatenate((features_CSP_test, test_extra_data_eeg), axis= 1)
print(all_final_features.shape)

In [ ]:
start = time.time()

clf = RandomForestClassifier(n_estimators=700,max_features='auto', random_state=0)
clf.fit(all_final_features, train_label)
print("trained")

labels_pred = clf.predict(all_final_features_test)
print("predicted")

cm = confusion_matrix(test_label, labels_pred)
acc = accuracy_score(test_label, labels_pred)
f1 = f1_score(test_label, labels_pred, average='macro')
print(cm, acc, f1)

print("time : ", time.time() - start)


In [ ]:
all_final_features = np.concatenate((features_CSP_train, train_extra_data), axis= 1)
all_final_features_test = np.concatenate((features_CSP_test, test_extra_data), axis= 1)
print(all_final_features.shape)

all_final_features = np.concatenate((all_final_features, train_extra_data_eeg), axis= 1)
all_final_features_test = np.concatenate((all_final_features_test, test_extra_data_eeg), axis= 1)
print(all_final_features.shape)


In [ ]:
start = time.time()

clf = RandomForestClassifier(n_estimators=700,max_features='auto', random_state=0)
clf.fit(all_final_features, train_label)
print("trained")

labels_pred = clf.predict(all_final_features_test)
print("predicted")

cm = confusion_matrix(test_label, labels_pred)
acc = accuracy_score(test_label, labels_pred)
f1 = f1_score(test_label, labels_pred, average='macro')
print(cm, acc, f1)

print("time : ", time.time() - start)


## Rieman

In [ ]:
riemann = riemannian_multiscale(filter_bank, time_windows, riem_opt=riem_opt, rho=rho, vectorized=True)
features_CSP_train_R = riemann.fit(train_data)
features_CSP_test_R = riemann.features(test_data)


In [ ]:
start = time.time()

clf = RandomForestClassifier(n_estimators=700,max_features='auto', random_state=0)
clf.fit(features_CSP_train_R, train_label)
print("trained")

labels_pred = clf.predict(features_CSP_test_R)
print("predicted")

cm = confusion_matrix(test_label, labels_pred)
acc = accuracy_score(test_label, labels_pred)
f1 = f1_score(test_label, labels_pred, average='macro')
print(cm, acc, f1)

print("time : ", time.time() - start)


In [ ]:

all_final_features = np.concatenate((features_CSP_train_R, train_extra_data), axis= 1)
all_final_features_test = np.concatenate((features_CSP_test_R, test_extra_data), axis= 1)
print(all_final_features.shape)

In [ ]:
start = time.time()

clf = RandomForestClassifier(n_estimators=700,max_features='auto', random_state=0)
clf.fit(all_final_features, train_label)
print("trained")

labels_pred = clf.predict(all_final_features_test)
print("predicted")

cm = confusion_matrix(test_label, labels_pred)
acc = accuracy_score(test_label, labels_pred)
f1 = f1_score(test_label, labels_pred, average='macro')
print(cm, acc, f1)

print("time : ", time.time() - start)


In [ ]:
all_final_features = np.concatenate((features_CSP_train_R, train_extra_data_eeg), axis= 1)
all_final_features_test = np.concatenate((features_CSP_test_R, test_extra_data_eeg), axis= 1)
print(all_final_features.shape)

In [ ]:
start = time.time()

clf = RandomForestClassifier(n_estimators=700,max_features='auto', random_state=0)
clf.fit(all_final_features, train_label)
print("trained")

labels_pred = clf.predict(all_final_features_test)
print("predicted")

cm = confusion_matrix(test_label, labels_pred)
acc = accuracy_score(test_label, labels_pred)
f1 = f1_score(test_label, labels_pred, average='macro')
print(cm, acc, f1)

print("time : ", time.time() - start)


In [ ]:
all_final_features = np.concatenate((features_CSP_train_R, train_extra_data), axis= 1)
all_final_features_test = np.concatenate((features_CSP_test_R, test_extra_data), axis= 1)
print(all_final_features.shape)

all_final_features = np.concatenate((all_final_features, train_extra_data_eeg), axis= 1)
all_final_features_test = np.concatenate((all_final_features_test, test_extra_data_eeg), axis= 1)
print(all_final_features.shape)


In [ ]:
start = time.time()

clf = RandomForestClassifier(n_estimators=700,max_features='auto', random_state=0)
clf.fit(all_final_features, train_label)
print("trained")

labels_pred = clf.predict(all_final_features_test)
print("predicted")

cm = confusion_matrix(test_label, labels_pred)
acc = accuracy_score(test_label, labels_pred)
f1 = f1_score(test_label, labels_pred, average='macro')
print(cm, acc, f1)

print("time : ", time.time() - start)


## Tout

In [ ]:
all_final_features = np.concatenate((features_CSP_train_R, train_extra_data), axis= 1)
all_final_features_test = np.concatenate((features_CSP_test_R, test_extra_data), axis= 1)
print(all_final_features.shape)

all_final_features = np.concatenate((all_final_features, train_extra_data_eeg), axis= 1)
all_final_features_test = np.concatenate((all_final_features_test, test_extra_data_eeg), axis= 1)
print(all_final_features.shape)

all_final_features = np.concatenate((all_final_features, features_CSP_train), axis= 1)
all_final_features_test = np.concatenate((all_final_features_test, features_CSP_test), axis= 1)
print(all_final_features.shape)



In [ ]:
start = time.time()

clf = RandomForestClassifier(n_estimators=700,max_features='auto', random_state=0)
clf.fit(all_final_features, train_label)
print("trained")

labels_pred = clf.predict(all_final_features_test)
print("predicted")

cm = confusion_matrix(test_label, labels_pred)
acc = accuracy_score(test_label, labels_pred)
f1 = f1_score(test_label, labels_pred, average='macro')
print(cm, acc, f1)

print("time : ", time.time() - start)
